# Exersise 1

Question 1.1: Revisiting SQL

Consider the following relations, where underlined attributes are primary keys, and attributes in
italics are foreign keys:

- Community(ZIP, CName)
- Person(Pid, PName, Age)
- Apartment(ApartNo, ZIP, Pid, Street, HouseNo)

Solve the following questions in SQL. Make sure to adhere to the given output schema if provided. For instance, the output schema (Pid, PName) is provided in e), requiring the result to
contain at least the mentioned attributes.

## a) 
Create the tables Community, Person, and Apartment. Assign useful data types for the corresponding attributes. Make sure that Age and HouseNo cannot be negative. Ensure the primary and foreign keys are appropriately set. Restrict every attribute from holding the NULL value, i.e., NULL values are not allowed. Finally, insert at least 5 tuples into each relation.

In [ ]:
CREATE TABLE
    Community (
        ZIP INT NOT NULL,
        CName TEXT NOT NULL,
        CONSTRAINT Community_pk PRIMARY KEY (ZIP)
    );

CREATE TABLE
    Person (
        Pid INT NOT NULL,
        PName TEXT NOT NULL,
        Age INT NOT NULL,
        CONSTRAINT Person_pk PRIMARY KEY (Pid),
        CONSTRAINT valid_age CHECK (Age > 0)
    );

CREATE TABLE
    Apartment (
        ApartNo INT NOT NULL,
        ZIP INT NOT NULL,
        Pid INT NOT NULL,
        Street TEXT NOT NULL,
        HouseNo INT NOT NULL,
        CONSTRAINT valid_house CHECK (HouseNo > 0),
        CONSTRAINT Apartment_pk PRIMARY KEY (ApartNo),
        CONSTRAINT Apartment_Community_fk FOREIGN KEY (ZIP) REFERENCES Community (ZIP),
        CONSTRAINT Apartment_Person_fk FOREIGN KEY (Pid) REFERENCES Person (Pid)
    );

INSERT INTO
    Community (ZIP, CName)
VALUES
    (10115, 'Berlin-Mitte'),
    (20095, 'Hamburg-Altstadt'),
    (50667, 'Cologne-Center'),
    (80331, 'Munich-Altstadt'),
    (60311, 'Frankfurt-Center');

INSERT INTO
    Person (Pid, PName, Age)
VALUES
    (1, 'Anna Schmidt', 29),
    (2, 'Max Müller', 35),
    (3, 'Sophie Keller', 22),
    (4, 'Jonas Becker', 41),
    (5, 'Lea Fischer', 27);

INSERT INTO
    Apartment (ApartNo, ZIP, Pid, Street, HouseNo)
VALUES
    (1001, 10115, 1, 'Unter den Linden', 12),
    (1002, 20095, 2, 'Mönckebergstraße', 45),
    (1003, 50667, 3, 'Hohe Straße', 7),
    (1004, 80331, 4, 'Marienplatz', 3),
    (1005, 60311, 5, 'Zeil', 88);

## b)
Create the table Land, and then delete it

In [ ]:
CREATE TABLE Land (
    id INT
);

DROP TABLE Land;

## c)
Increase the age of every person that is older than 18 by one.

In [ ]:
UPDATE Person
SET
    age = age + 1
WHERE
    age > 18;

## d)
Delete every apartment in the street Bahnhofstraße with ZIP code 35037.

In [ ]:
DELETE FROM Apartment
WHERE
    Street = 'Bahnhofstraße'
    AND ZIP = 35037;

## e) 
Who is older than 18?
###### Schema: (Pid, PName)

In [ ]:
SELECT Pid, PName
FROM
    Person
WHERE
    Age > 18;

## f)
For which community names is there at least one apartment in the street Hauptstraße?
###### Schema: (CName)

In [ ]:
SELECT
    c.CName AS CName
FROM
    Community c
WHERE
    EXISTS (
        SELECT
            1
        FROM
            Apartment a
        WHERE
            a.ZIP = c.ZIP
            AND a.Street = 'Hauptstraße'
    );

## g) 
How many apartments are there in the community named Marburg?
###### Schema: (NumberOfApartments)

In [ ]:
SELECT
    COUNT(*) AS NumberOfApartments
FROM
    Apartment a
WHERE
    a.ZIP IN (
        SELECT
            c.ZIP
        FROM
            Community c
        WHERE
            c.CName = 'Marburg'
    );

## h) 
How old are the people owning apartments on average per community? Sort the output in
ascending order by average age.
###### Schema: (ZIP, AverageAge)

In [ ]:
SELECT
    a.ZIP,
    AVG(p.Age) AS AverageAge
FROM
    Apartment a
JOIN
    Person p
ON
    a.Pid = p.Pid
GROUP BY
    a.ZIP
ORDER BY
    AVG(p.Age);

## i)
Who owns an above-average number of apartments?
###### Schema: (Pid, PName)

In [ ]:
SELECT
    p.Pid,
    p.PName,
    COUNT(a.ApartNo) AS NumApartments
FROM
    Person p
    JOIN Apartment a ON p.Pid = a.Pid
GROUP BY
    p.Pid,
    p.PName
HAVING
    COUNT(a.ApartNo) > (
        SELECT
            AVG(apartment_count)
        FROM
            (
                SELECT
                    COUNT(*) AS apartment_count
                FROM
                    Apartment
                GROUP BY
                    Pid
            ) sub
    )
ORDER BY
    NumApartments DESC;

# Question 1.2: Geometries
This question requires you to implement various geometric structures and functionalities in
PostgreSQL without PostGIS

## a) 
Create a new type named PointType. This type represents a point comprising two coordinates x and y, which are of type NUMERIC.

## b)
Create a new table Rectangle to store axis-aligned rectangles. An axis-aligned rectangle is identified by its primary key id, and contains two points of type PointType. Ensure the following constraints are satisfied:
- The first point is the bottom left corner of the rectangle.
- The second point is the top right corner of the rectangle.
- A rectangle can be created using the two points given.

## c)
Create a new table named Triangle. A triangle is identified by its primary key id, and contains three points of type PointType. Ensure a valid triangle can be created using the three points provided, i.e., they do not degenerate into a line.

## d)
Connect to your database using Python. Solve the following questions using python with the psycopg package and a jupyter-notebook (You can pass SQL code as string to the curser.execute() function.
- Insert at least five tuples into each relation Rectangle and Triangle. Make sure you insert at least one square, one right-angled triangle, and one isosceles triangle.
- Query all rectangles. Iterate through the result, and if the current rectangle is a square, print it to the console.
- Query all triangles. Iterate through the result, and if the current triangle is right-angled or isosceles, print it to the console.

## e)
Implement the following topological relationships of two rectangles r1 and r2 in Python:
- equals(r1, r2): checks whether the rectangle r1 equals the rectangle r2, i.e., it checks if the points in r1 are equal to the ones in r2.
- inside(r1, r2): checks if the rectangle r1 is contained in the rectangle r2.
- intersects_not_inside(r1, r2): checks if rectangles r1 and r2 intersect. Note that in case a rectangle r1 is contained in another rectangle r2 or vice-versa, then this method returns False.
- disjoint(r1,r2): checks whether the two rectangles r1, and r2 are disjoint.